In [0]:
#Let’s get started by reading the data set into a temporary view
from pyspark.sql import SparkSession 
# Create a SparkSession
spark = (SparkSession
 .builder
 .appName("SparkSQLExampleApp")
 .getOrCreate())
# Path to data set
csv_file = "/FileStore/tables/departuredelays.csv"
# Read and create a temporary view
# Infer schema (note that for larger files you 
# may want to specify the schema)
df = spark.read\
          .option("header", "true")\
          .option("inferSchema", "true")\
          .csv(csv_file)
df.createOrReplaceTempView("us_delay_flights_tbl")
#Now that we have a temporary view, we can issue SQL queries using Spark SQL

In [0]:
df.show()

+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1011245|    6|     602|   ABE|        ATL|
|1020600|   -8|     369|   ABE|        DTW|
|1021245|   -2|     602|   ABE|        ATL|
|1020605|   -4|     602|   ABE|        ATL|
|1031245|   -4|     602|   ABE|        ATL|
|1030605|    0|     602|   ABE|        ATL|
|1041243|   10|     602|   ABE|        ATL|
|1040605|   28|     602|   ABE|        ATL|
|1051245|   88|     602|   ABE|        ATL|
|1050605|    9|     602|   ABE|        ATL|
|1061215|   -6|     602|   ABE|        ATL|
|1061725|   69|     602|   ABE|        ATL|
|1061230|    0|     369|   ABE|        DTW|
|1060625|   -3|     602|   ABE|        ATL|
|1070600|    0|     369|   ABE|        DTW|
|1071725|    0|     602|   ABE|        ATL|
|1071230|    0|     369|   ABE|        DTW|
|1070625|    0|     602|   ABE|        ATL|
|1071219|    0|     569|   ABE|        ORD|
|1080600|    0|     369|   ABE| 

In [0]:
#find all flights whose distance is greater than 1,000 miles
spark.sql(""" SELECT distance, origin, destination
 FROM us_delay_flights_tbl WHERE distance > 1000
 ORDER BY distance DESC """).show(10)

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows



In [0]:
#find all flights between San Francisco (SFO) and Chicago (ORD) with at least a two-hour delay
spark.sql(""" SELECT delay, origin, destination
FROM us_delay_flights_tbl 
WHERE delay > 120 AND origin = 'SFO' AND destination = 'ORD'
ORDER BY delay DESC""").show(10)

+-----+------+-----------+
|delay|origin|destination|
+-----+------+-----------+
| 1638|   SFO|        ORD|
|  396|   SFO|        ORD|
|  326|   SFO|        ORD|
|  320|   SFO|        ORD|
|  297|   SFO|        ORD|
|  296|   SFO|        ORD|
|  279|   SFO|        ORD|
|  274|   SFO|        ORD|
|  266|   SFO|        ORD|
|  258|   SFO|        ORD|
+-----+------+-----------+
only showing top 10 rows



In [0]:
#we want to label all US flights, regardless of origin and destination, with an indication of the delays they experienced: Very Long Delays (> 6 hours), Long Delays (2–6 hours), etc. We’ll add these human-readable labels in a new column called Flight_Delays
spark.sql(""" SELECT delay, origin, destination,
CASE 
    WHEN delay > 360 THEN 'Very Long Delays'
    WHEN delay >120 AND delay < 360 THEN 'Long Delays'
    WHEN delay > 60 AND delay < 120 THEN 'Short Delays'
    WHEN delay > 0 and delay < 60 THEN 'Tolerable Delays'
    WHEN delay = 0 THEN 'No Delays'
    ELSE 'Early'
END AS Flight_Delays
FROM us_delay_flights_tbl 
ORDER BY origin, delay DESC """).show(10)


+-----+------+-----------+-------------+
|delay|origin|destination|Flight_Delays|
+-----+------+-----------+-------------+
|  333|   ABE|        ATL|  Long Delays|
|  305|   ABE|        ATL|  Long Delays|
|  275|   ABE|        ATL|  Long Delays|
|  257|   ABE|        ATL|  Long Delays|
|  247|   ABE|        ATL|  Long Delays|
|  247|   ABE|        DTW|  Long Delays|
|  219|   ABE|        ORD|  Long Delays|
|  211|   ABE|        ATL|  Long Delays|
|  197|   ABE|        DTW|  Long Delays|
|  192|   ABE|        ORD|  Long Delays|
+-----+------+-----------+-------------+
only showing top 10 rows



In [0]:
#To read Parquet files into a DataFrame, you simply specify the format and path
file = """/FileStore/tables/part_r_00000_1a9822ba_b8fb_4d8e_844a_ea30d0801b9e_gz.parquet"""
df = spark.read.format("parquet").load(file)
df.show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|    1|
|       United States|            Ireland|  264|
|       United States|              India|   69|
|               Egypt|      United States|   24|
|   Equatorial Guinea|      United States|    1|
|       United States|          Singapore|   25|
|       United States|            Grenada|   54|
|          Costa Rica|      United States|  477|
|             Senegal|      United States|   29|
|       United States|   Marshall Islands|   44|
|              Guyana|      United States|   17|
|       United States|       Sint Maarten|   53|
|               Malta|      United States|    1|
|             Bolivia|      United States|   46|
|            Anguilla|      United States|   21|
|Turks and Caicos ...|      United States|  136|
|       United States|        Afghanistan|    2|
|Saint Vincent and..

In [0]:
#Reading Parquet files into a Spark SQL table
CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl
    USING parquet
    OPTIONS (
        path "/FileStore/tables/part_r_00000_1a9822ba_b8fb_4d8e_844a_ea30d0801b9e_gz.parquet" )

  File "<command-3863854355457319>", line 2
    CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl
           ^
SyntaxError: invalid syntax


In [0]:
spark.sql("SELECT * FROM us_delay_flights_tbl").show()


+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1011245|    6|     602|   ABE|        ATL|
|1020600|   -8|     369|   ABE|        DTW|
|1021245|   -2|     602|   ABE|        ATL|
|1020605|   -4|     602|   ABE|        ATL|
|1031245|   -4|     602|   ABE|        ATL|
|1030605|    0|     602|   ABE|        ATL|
|1041243|   10|     602|   ABE|        ATL|
|1040605|   28|     602|   ABE|        ATL|
|1051245|   88|     602|   ABE|        ATL|
|1050605|    9|     602|   ABE|        ATL|
|1061215|   -6|     602|   ABE|        ATL|
|1061725|   69|     602|   ABE|        ATL|
|1061230|    0|     369|   ABE|        DTW|
|1060625|   -3|     602|   ABE|        ATL|
|1070600|    0|     369|   ABE|        DTW|
|1071725|    0|     602|   ABE|        ATL|
|1071230|    0|     369|   ABE|        DTW|
|1070625|    0|     602|   ABE|        ATL|
|1071219|    0|     569|   ABE|        ORD|
|1080600|    0|     369|   ABE| 

In [0]:
#Writing DataFrames to Parquet files
df.write.format("parquet")\
  .mode("overwrite")\
  .option("compression", "snappy")\
  .save("/FileStore/tables/dfparquet")



In [0]:
#Writing DataFrames to Spark SQL tables
df.write\
  .mode("overwrite")\
  .saveAsTable("us_delay_flights_tbl")


In [0]:
#GlobalTempView vs TempView
#Las vistas TEMPORALES están en el ámbito de la spark session y se eliminarán cuando finalice la sesión.ç
#Las vistas TEMPORALES GLOBALES están vinculadas a una base de datos temporal preservada por el sistema global_temp . Crea una vista si no existe.